## Model building and training file

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

from keras.initializers import RandomNormal
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation, Dense, Dropout, Flatten
from keras.layers.pooling import MaxPooling2D

from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


## Define few paths

In [2]:
# Define few paths
data_dir = './data'
train_data_dir = './data/train'
test_data_dir = './data/test'

In [34]:
batch_size = 32
dim_x = 224
dim_y = 224
n_channels = 3
n_classes = 120
n_samples = 10222
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(dim_x, dim_y),  # all images will be resized
        batch_size=batch_size,
        class_mode='categorical')

Found 10222 images belonging to 120 classes.


In [35]:
def get_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(dim_x, dim_y, n_channels))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(units=n_classes,
              kernel_initializer=RandomNormal(mean=0.0, stddev=0.01),
              bias_initializer='zeros',
              kernel_regularizer=l2(0.001),
              bias_regularizer=None)(x)

    predictions = Activation('softmax')(x)

    # create graph of new model
    model = Model(inputs=base_model.input, outputs=predictions)

    # freeze all convolutional base model layers
    for layer in base_model.layers:
        layer.trainable = False

    return model

In [36]:
# get the model
model = get_model()
# define the checkpoint
file_path = "best_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [38]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=2,
        callbacks=callbacks_list,
        verbose=2)

Epoch 1/2
Epoch 00001: loss improved from inf to 0.05262, saving model to best_model.h5
 - 1114s - loss: 0.0526 - acc: 0.9917
Epoch 2/2
Epoch 00002: loss improved from 0.05262 to 0.04585, saving model to best_model.h5
 - 1096s - loss: 0.0459 - acc: 0.9917
